In [17]:
import pandas as pd
import sklearn
import sklearn.metrics

In [37]:
# Using only split 0

SPLITS = [0,1,2,3]

wsi_data = pd.concat([pd.read_csv(f"D:\data\Outputs\WSI\PDAC_train_0_{SPLIT}.csv") for SPLIT in SPLITS]) # CHANGE THE SECOND INDEX

data = [wsi_data, pd.read_csv(f"D:\data\Outputs\RNASeq\PDAC_train_1.csv")]

data[1]["Sample ID"] =  data[1]["Sample ID"].apply(lambda x: x.replace('.', '-') if len(x.split('.')) == 3 else x.replace('.', '-')[:-2])
 

sample_ids = list(set(data[0]["Sample ID"]).intersection(data[1]["Sample ID"]))

# Dropping all data that is not in both sets
for dat in data:
    for column in dat:
        if column not in sample_ids:
            dat.drop(column, axis="columns")

data = pd.merge(data[0], data[1], on='Sample ID')
data = data.dropna()

In [38]:
labels = data["Label_x"]
out = round((data["Output_x"] + data["Output_y"])/2)

print(labels,out)

0     1
1     1
2     1
3     1
4     0
5     1
6     1
7     1
8     1
9     1
10    1
11    1
12    1
13    0
14    1
15    1
16    1
17    1
18    1
19    1
20    1
21    1
22    0
23    1
24    0
25    1
26    1
27    0
28    1
29    1
30    1
31    0
32    1
33    1
34    1
35    1
36    1
37    1
38    0
39    1
40    1
Name: Label_x, dtype: int64 0     0.0
1     0.0
2     0.0
3     0.0
4     1.0
5     0.0
6     0.0
7     0.0
8     0.0
9     0.0
10    0.0
11    0.0
12    0.0
13    1.0
14    0.0
15    0.0
16    0.0
17    0.0
18    0.0
19    0.0
20    0.0
21    0.0
22    1.0
23    0.0
24    1.0
25    0.0
26    0.0
27    1.0
28    0.0
29    0.0
30    0.0
31    1.0
32    0.0
33    0.0
34    0.0
35    0.0
36    0.0
37    0.0
38    1.0
39    0.0
40    0.0
dtype: float64


In [39]:
conf_m = sklearn.metrics.confusion_matrix(labels, out, labels=[0, 1])
acc = sklearn.metrics.accuracy_score(labels, out)
f1 = sklearn.metrics.f1_score(labels, out)

print(acc)
print(f1)
print(conf_m)

0.0
0.0
[[ 0  7]
 [34  0]]


In [ ]:
def preprocess(train_data, test_data):
    train_sample_ids, test_sample_ids = train_data["Unnamed: 0"], test_data["Unnamed: 0"]
    train_genes, test_genes = train_data.columns[1:-1], test_data.columns[1:-1]

    genes = list(set(train_genes).intersection(test_genes))

    for column in test_data:
        if column not in genes:
            test_data.drop(column, axis="columns")

    for column in train_data:
        if column not in genes:
            train_data.drop(column, axis="columns")

    train_X, test_X = [list(row[1][genes]) for row in train_data.iterrows()], [list(row[1][genes]) for row in test_data.iterrows()]
    train_y, test_y = [0 if dat=="Solid Tissue Normal" else 1 for dat in train_data["Labels"]], [0 if dat=="Solid Tissue Normal" else 1 for dat in test_data["Labels"]]

    #test_X, test:y = sklearn.utils.shuffle(test_X, test_y, random_state=0)
    
    return train_X, train_y, test_X, test_y, train_sample_ids, test_sample_ids